<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#I.-Обзор-и-обработка-данных" data-toc-modified-id="I.-Обзор-и-обработка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><strong>I. Обзор и обработка данных</strong></a></span><ul class="toc-item"><li><span><a href="#1.-Первичный-обзор" data-toc-modified-id="1.-Первичный-обзор-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1. Первичный обзор</a></span></li><li><span><a href="#2.-Лемматизация-текста" data-toc-modified-id="2.-Лемматизация-текста-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>2. Лемматизация текста</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span><strong>Выводы</strong></a></span></li></ul></li><li><span><a href="#II.-Сплит-данных-на-выборки" data-toc-modified-id="II.-Сплит-данных-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><strong>II. Сплит данных на выборки</strong></a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><strong>Выводы</strong></a></span></li></ul></li><li><span><a href="#III.-Построение-моделей" data-toc-modified-id="III.-Построение-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><strong>III. Построение моделей</strong></a></span><ul class="toc-item"><li><span><a href="#1.-Модель-«Случайный-лес»" data-toc-modified-id="1.-Модель-«Случайный-лес»-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>1. Модель «Случайный лес»</a></span></li><li><span><a href="#2.-Модель-«Логистическая-регрессия»" data-toc-modified-id="2.-Модель-«Логистическая-регрессия»-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>2. Модель «Логистическая регрессия»</a></span></li><li><span><a href="#3.-Модель-LightGBM" data-toc-modified-id="3.-Модель-LightGBM-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>3. Модель LightGBM</a></span></li><li><span><a href="#4.-Модель-XGBoost" data-toc-modified-id="4.-Модель-XGBoost-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>4. Модель XGBoost</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#III.-Проверка-моделей-на-тестовой-выборке" data-toc-modified-id="III.-Проверка-моделей-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><strong>III. Проверка моделей на тестовой выборке</strong></a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#IV.-Проверка-моделей-на-вменяемость" data-toc-modified-id="IV.-Проверка-моделей-на-вменяемость-5"><span class="toc-item-num">5&nbsp;&nbsp;</span><strong>IV. Проверка моделей на вменяемость</strong></a></span></li><li><span><a href="#V.-Итоги-исследования" data-toc-modified-id="V.-Итоги-исследования-6"><span class="toc-item-num">6&nbsp;&nbsp;</span><strong>V. Итоги исследования</strong></a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span><strong>Выводы</strong></a></span></li></ul></li></ul></div>

In [2]:
#pip list

In [1]:
#!pip3 install -U spacy
#!pip3 install -U spacy-lookups-data
#!python3 -m spacy download en_core_web_sm
# !pip install hyperopt
#!pip install xgboost

     |████████████████████████████████| 1.6 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 80.0 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 60.0 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=976534e2ae2b965a1af020dab785d735448bfe6ca9ed1ed672974d77677fc784
  Stored in directory: /home/jovyan/.cache/pip/wheels/2f/a0/d3/4030d9f80e6b3be787f19fc911b8e7aa462986a40ab1e4bb94
Successfully built future


In [3]:
import os
import re
import time
import warnings
warnings.filterwarnings('ignore')
import math

from tqdm.auto import tqdm
tqdm.pandas()

import pandas as pd
import numpy as np

import spacy
disabled_pipes = [ "parser",  "ner"]
nlp = spacy.load('en_core_web_sm', disable=disabled_pipes)

import nltk
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
stop_words = set(nltk_stopwords.words('english'))

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
import xgboost as xgb
from sklearn.dummy import DummyClassifier

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Проект для «Викишоп»

**Заказчик** — интернет-магазин «Викишоп».

**Цель исследования**:
1. Построить систему, способную давать оценку эмоциональной окраске предлагаемого клиентом комментария.

**Поставленные задачи**
1. Изучить предоставленные данные и подготовить их для обучения моделей.
2. Обучить разные модели и найти лучшую с предельно высоким значением F1-меры. Минимальный показатель метрики **желательно** 0.75.
3. Проверить качество выбранной модели на тестовой выборке.

**Дальнейшее использование**

Результаты исследования будут использоваться в дальнейшем при модерации токсичных комментариев, добавляемых к описанию товаров, и для допуска к их публикации.

**Дополнительно предоставленная инфомарция**

- Имеется набор данных с разметкой о токсичности правок.
- Ключевые показатели моделей:
    * качество предсказания;
    * скорость предсказания;
    * время обучения.
- Модель будет строиться без использоватья `BERT` из-за ограниченных возможностей Jupyter Notebook в 4 Гб оперативной памяти.

## **I. Обзор и обработка данных**

### 1. Первичный обзор
Сначала составим представление о полученных исходных данных.

Прочитаем файл `toxic_comments.csv` из папки `/datasets/` и сохраним его в переменной `df`:

In [4]:
pth1 = 'D:/Programs/Jupyter_projects/datasets/toxic_comments.csv'
pth2 = '/datasets/toxic_comments.csv'

if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
else:
    print('Something is wrong')

Выведем на экран первые пять строк таблицы и получим общую информацию:

In [5]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Итак, в таблице 2 столбца. Типы данных в столбцах — объекты (строки) `object` и целые числа `int64`.

Согласно документации к данным:

* `text` — текст оставленного комментария
* `toxic` — эмоциональная оценка: отрицательная - 1 (токсичный комментарий), положительная - 0 (нейтральный/положительный)

Количество значений в столбцах соотнесены полностью. Значит, в данных нет **пропущенных значений**.

### 2. Лемматизация текста

Выведем размер классов

In [7]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Видно, что классы перекошены, соответственно будем их в дальнейшем балансировать.

Напишем две функции: очистки текста от пунктуации и лишних знаков (для ускорения лемматизации) и для поиска лемм

In [8]:
# лемматизация с помощью библиотеки SpaCy
def lemmatize(text):
    doc = nlp(text)
    lemm_list = []
    
    for token in doc:
        lemm_list.append(token.lemma_)
    lemm_text = " ".join(lemm_list)
    
    return lemm_text


# очистка текста
def clear_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    
    
    return ' '.join(re.sub(r'([^a-zA-Z ])()', ' ', text).split())

Применим написанные функции к части исходного датафрейма для создания столбца с очищенным текстом и с леммами

In [9]:
df['text_clear'] = df['text'].map(lambda x: clear_text(x))

df['text_lemm'] = df['text_clear'].progress_apply(lemmatize)

df.head(3)

  0%|          | 0/159571 [00:00<?, ?it/s]

,text,toxic,text_clear,text_lemm
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...,explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i am s...,d aww he match this background colour I be see...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i am really not trying to edit war it ...,hey man I be really not try to edit war it jus...


### **Выводы**

Каждый объект в наборе данных — уникальный комментарий пользователя сайта, оставленный в качестве описания к товару. 
В ходе подготовки данных текстовая часть была почищена и лемматизирована для анализа и обучения.

## **II. Сплит данных на выборки**

Предоставленные данные представляют собой один единый датафрейм, разделения на обучающую, тестовую и валидационную выборки у нас сейчас нет. Поэтому выполним это сейчас с помощью алгоритма `train_test_split`.

Целевой признак - токсичность комментария

In [10]:
features = df.drop('toxic', axis = 1)
target = df['toxic']

In [11]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size = 0.2, random_state=123456)

Преобразуем выборки в корпусы

In [12]:
corpus_train = features_train['text_lemm'].values #.astype('U')

corpus_test = features_test['text_lemm'].values #.astype('U')

In [13]:
# добавим все выборки в словарь
arrays = {
    'обучающая' : corpus_train,
    'тестовая' : corpus_test}

# выведем количество содержащихся объектов
for array_name, array in arrays.items():
    print("\033[4m\033[1m\033[31m{}\033[0m содержит объектов:".format(f'{array_name} выборка'), 
          array.shape[0], f'({array.shape[0]/df.shape[0]:.1%})')

обучающая выборка содержит объектов: 127656 (80.0%)
тестовая выборка содержит объектов: 31915 (20.0%)


Создадим матрицы признаков для выборок для дальнейшего определения тональности. В качестве целевого признака укажем определение токсичности комментария

In [14]:
# фитим обучающую выборку
count_tf_idf = TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stop_words) 
tf_idf_fitted = count_tf_idf.fit(corpus_train)

In [15]:
# трансформируем
tf_idf_train = tf_idf_fitted.transform(corpus_train)
print("Размер обучающей матрицы:", tf_idf_train.shape)

tf_idf_test = tf_idf_fitted.transform(corpus_test)
print("Размер тестовой матрицы:", tf_idf_test.shape)

Размер обучающей матрицы: (127656, 304588)
Размер тестовой матрицы: (31915, 304588)


### **Выводы**

- Создали две выборки: тестовую и обучающую. Преобразовали их в матрицы на основе TF-IDF для дальнейшего обучения.

Также с целью выравнивания баланса классов были испробованы методы:
- **взвешивания классов `class_weight = 'balanced'`- вариант, на котором остановился**
- увеличения выборки редких значений путем их дублирования `upsample`
- уменьшения выборки путем удаления частых значений `downsample` 

## **III. Построение моделей**

### 1. Модель «Случайный лес»

In [16]:
space = {'max_depth': hp.choice ('max_depth', np.arange(1, 100+1)),
        'n_estimators' : hp.choice ('n_estimators', np.arange(1, 50, dtype=int)),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        }                        
                                   
def best_par_hyperopt(features_train, target_train):

    features_subtrain, features_valid, target_subtrain, target_valid = train_test_split(
    features_train, target_train, test_size=0.25, random_state = 12345)  
    
    def objective(space):
        
        model_opt = RandomForestClassifier(random_state=12345, 
                                      max_depth = space['max_depth'], 
                                      n_estimators = space['n_estimators'],
                                      criterion = 'entropy',
                                      min_samples_split = space['min_samples_split'],
                                      class_weight = 'balanced'
                                     )
    
        model_opt.fit(features_subtrain, target_subtrain)
        
        predictions = model_opt.predict(features_valid)
        accuracy = f1_score(target_valid, predictions)

        return {'loss': -accuracy, 'status': STATUS_OK }
    
    trials = Trials()
    best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 100,
            trials= trials,
            show_progressbar=True)
    
    print(best)

    # засекаем время на обучение лучшей модели
    start_time = time.time()
    
    #лучшую модель будем обучать уже на объединенной обучающей и валидационной
    best_model = RandomForestClassifier(random_state=12345, 
                                        max_depth = best['max_depth'], 
                                        n_estimators = best['n_estimators'],
                                        criterion = 'entropy',
                                        min_samples_split = best['min_samples_split'],
                                        class_weight = 'balanced'
                                       )

    best_model.fit(features_subtrain, target_subtrain)
        
    predictions = best_model.predict(features_valid)
    accuracy_best = round(f1_score(target_valid, predictions), 3)
     
    model_time = round((time.time() - start_time), 5)
    
    print(f'Время выполнения обучения c с лучшими параметрами составляет {model_time:.3f} секунд')

    return best, model_time, accuracy_best, best_model

In [17]:
best_forest_param, best_forest_time, best_forest_f1, best_forest_model = (
    best_par_hyperopt(tf_idf_train, target_train))
print('Значение F1-меры равно', best_forest_f1)

100%|██████████| 100/100 [10:42<00:00,  6.43s/trial, best loss: -0.5001015228426395]
{'max_depth': 99, 'min_samples_split': 0.0032991330999791, 'n_estimators': 34}
Время выполнения обучения c с лучшими параметрами составляет 27.439 секунд
Значение F1-меры равно 0.495


### 2. Модель «Логистическая регрессия»

In [18]:
%%time

model_LR = LogisticRegression(random_state=12345)

params = {'C': [0.1, 5, 10, 15, 20]}

g_cv_LR = GridSearchCV(estimator=model_LR, param_grid=params, cv=3, scoring='f1', n_jobs=-1, refit=False)
g_cv_LR.fit(tf_idf_train, target_train)

model_LR_params_best = g_cv_LR.best_params_
best_LR_f1 = g_cv_LR.best_score_

print(model_LR_params_best)
print(g_cv_LR.best_score_)

{'C': 20}
0.7622279510067863
CPU times: user 5min 14s, sys: 9min 12s, total: 14min 26s
Wall time: 14min 27s


In [32]:
start_time = time.time()
model_LR_cv = LogisticRegression(random_state=12345, solver='lbfgs', C=20, class_weight = 'balanced')
model_LR_cv.fit(tf_idf_train, target_train)

best_LR_cv_f1 = cross_val_score(model_LR_cv, tf_idf_train, target_train, scoring="f1").mean()

model_LR_cv_time = round((time.time() - start_time), 5)

print('Значение F1-меры равно', best_LR_cv_f1)

Значение F1-меры равно 0.7874250584681066


In [24]:
start_time = time.time()
model_LR_ncv = LogisticRegression(random_state=12345, solver='lbfgs', C=20, class_weight = 'balanced')

tf_idf_subtrain, tf_idf_valid, target_subtrain, target_valid = train_test_split(
tf_idf_train, target_train, test_size=0.25, random_state = 12345)  

model_LR_ncv.fit(tf_idf_subtrain, target_subtrain)
predictions_LR_ncv = model_LR_ncv.predict(tf_idf_valid)

model_LR_ncv_time = round((time.time() - start_time), 5)

best_LR_ncv_f1 = round(f1_score(target_valid, predictions_LR_ncv), 5)
print('Значение F1-меры равно', best_LR_ncv_f1)

Значение F1-меры равно 0.77861


In [25]:
test_predictions_LR_ncv = model_LR_ncv.predict(tf_idf_test)
accuracy_test_LR_ncv = round(f1_score(target_test, test_predictions_LR_ncv), 3)
    
print(f'F1-мера модели на тестовой выборке:', accuracy_test_LR_ncv)
print()

F1-мера модели на тестовой выборке: 0.788



### 3. Модель LightGBM

In [26]:
space = {'max_depth': hp.choice ('max_depth', np.arange(1, 100+1, dtype=int)),
        'n_estimators' : hp.choice ('n_estimators', np.arange(1, 100+1, dtype=int))
    }

def best_par_hyperopt(features_train, target_train):

    features_subtrain, features_valid, target_subtrain, target_valid = train_test_split(
    features_train, target_train, test_size=0.25, random_state = 12345)      
    
    def objective(space):
        
        model_opt = LGBMClassifier(random_state=12345, 
                                      max_depth = space['max_depth'], 
                                      n_estimators = space['n_estimators'],
                                      class_weight = 'balanced'
                                     )
    
        model_opt.fit(features_subtrain, target_subtrain, verbose=0)
        
        predictions = model_opt.predict(features_valid)
        accuracy = f1_score(target_valid, predictions)

        return {'loss': -accuracy, 'status': STATUS_OK }
    
    trials = Trials()
    best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 100,
            trials= trials,
            show_progressbar=True)
    
    print(best)

    # засекаем время на обучение лучшей модели
    start_time = time.time()
    
    #лучшую модель будем обучать уже на объединенной обучающей и валидационной
    best_model = LGBMClassifier(random_state=12345, 
                                max_depth = best['max_depth'], 
                                n_estimators = best['n_estimators'],
                                class_weight = 'balanced')
    
    best_model.fit(features_subtrain, target_subtrain, verbose=0)
   
    model_time = round((time.time() - start_time), 5)
    
    predictions = best_model.predict(features_valid)
    accuracy_best = round(f1_score(target_valid, predictions), 3)
    
    print(f'Время выполнения обучения c с лучшими параметрами составляет {model_time:.3f} секунд')
    
    return best, model_time, accuracy_best, best_model

In [27]:
best_LGBM_param, best_LGBM_time, best_LGBM_f1, best_LGBM_model = (
    best_par_hyperopt(tf_idf_train, target_train))
print('Значение F1-меры равно', best_LGBM_f1)

100%|██████████| 100/100 [3:44:13<00:00, 134.54s/trial, best loss: -0.7365070177977139] 
{'max_depth': 80, 'n_estimators': 86}
Время выполнения обучения c с лучшими параметрами составляет 161.037 секунд
Значение F1-меры равно 0.736


### 4. Модель XGBoost

In [33]:
space={'max_depth': hp.choice('max_depth', np.arange(1, 100+1, 2, dtype=int)),
        'n_estimators' : hp.choice('n_estimators', np.arange(1, 100+1, 2, dtype=int))
      }

def best_par_hyperopt(features_train, target_train):

    features_subtrain, features_valid, target_subtrain, target_valid = train_test_split(
    features_train, target_train, test_size=0.25, random_state = 12345)          
    
    def objective(space):
        
        model_opt = xgb.XGBClassifier(random_state=12345, 
                                      max_depth = space['max_depth'], 
                                      n_estimators = space['n_estimators'],
                                      eval_metric='logloss'
                                     )
    
        model_opt.fit(features_subtrain, target_subtrain)
        
        predictions = model_opt.predict(features_valid)
        accuracy = f1_score(target_valid, predictions)

        return {'loss': -accuracy, 'status': STATUS_OK }
    
    trials = Trials()
    best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 50,
            trials= trials,
            show_progressbar=True)
    
    print(best)

    # засекаем время на обучение лучшей модели
    start_time = time.time()
    
    best_model = xgb.XGBClassifier(random_state=12345, 
                                   max_depth = best['max_depth'],
                                   eval_metric='logloss',
                                   n_estimators = best['n_estimators']
                                  )
    
    best_model.fit(features_subtrain, target_subtrain)
    
    model_time = round((time.time() - start_time), 5)
    
    predictions = best_model.predict(features_valid)
    accuracy_best = round(f1_score(target_valid, predictions), 3)
    
    print(f'Время выполнения обучения c с лучшими параметрами составляет {model_time:.3f} секунд')
    
    return best, model_time, accuracy_best, best_model

In [29]:
#best_XGB_param, best_XGB_time, best_XGB_f1, best_XGB_model = (
#    best_par_hyperopt(tf_idf_train, target_train))
#print('Значение F1-меры равно', best_XGB_f1)

[01:22:58] WARNING: ../src/learner.cc:576:            
Parameters: { "class_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:24:56] WARNING: ../src/learner.cc:576:                                          
Parameters: { "class_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:34:00] WARNING: ../src/learner.cc:576:                                          
Parameters: { "class_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings 

KeyboardInterrupt: 

<div class="alert alert-info">

### <span style="color: ##4682b4; font-weight: 600">Комментарий </span> <a class="tocSkip">
     
очень долгое выполнение модели... остановил после 4,5 часов при лучшем результате 

46%|████▌     | 23/50 [4:36:32<7:59:24, 1065.36s/trial, best loss: -0.7607340103331551]

Даже из коробки не смогла запуститься... По всей видимости, слишком много признаков и объектов для обучения, юпитер не вывозит быстрое обучение)

</div>

start_time = time.time()
best_XGB_model = xgb.XGBClassifier(random_state=12345, 
                                   #max_depth = 80, 
                                   #n_estimators = 80,
                                   eval_metric='logloss'
                                  )

best_XGB_model.fit(tf_idf_train, target_train)

best_XGB_f1 = cross_val_score(best_XGB_model, tf_idf_train, target_train, scoring="f1").mean()
        
best_XGB_time = round((time.time() - start_time), 5)

print('Значение F1-меры равно', best_XGB_f1)

### Выводы

Получили следующие данных по лучшим моделям на обучении

In [46]:
models_description = [
    ['Случайный лес', best_forest_f1, best_forest_time, best_forest_param],
    ['Логистическая регрессия с CV', best_LR_cv_f1, model_LR_cv_time, ["'C' : 20", "'class_weight' : 'balanced'"]],
    ['Логистическая регрессия без CV', best_LR_ncv_f1, model_LR_ncv_time, ["'C' : 20", "'class_weight' : 'balanced'"]],
    #['XGBoost', best_XGB_f1, best_XGB_time, best_XGB_param],
    ['LightGBM', best_LGBM_f1, best_LGBM_time, best_LGBM_param],
]

columns_models_description = ['Модель', 'Лучшее качество F1-меры при обучении', 
                              'Время выполнения оптимального параметра, сек.', 'Лучшие гиперпараметры']

models_summary = pd.DataFrame(data=models_description, columns=columns_models_description)
models_summary.sort_values(by='Лучшее качество F1-меры при обучении', ascending=False)

,Модель,Лучшее качество F1-меры при обучении,"Время выполнения оптимального параметра, сек.",Лучшие гиперпараметры
1,Логистическая регрессия с CV,0.787425,385.77596,"['C' : 20, 'class_weight' : 'balanced']"
2,Логистическая регрессия без CV,0.778610,60.79804,"['C' : 20, 'class_weight' : 'balanced']"
3,LightGBM,0.736000,161.03665,"{'max_depth': 80, 'n_estimators': 86}"
0,Случайный лес,0.495000,27.43883,"{'max_depth': 99, 'min_samples_split': 0.00329..."


## **III. Проверка моделей на тестовой выборке**

Проверим наши лучшие модели на тестовой выбоке, чтобы убедиться в их качестве

In [47]:
# добавим все лучшие модели в словарь для быстрого доступа к названию и самой модели
models = {'Случайный лес' : best_forest_model,
          'Логистическая регрессия с CV' : model_LR_cv,
          'Логистическая регрессия без CV' : model_LR_ncv,
          #'XGBoost' : best_XGB_model,
          'LightGBM' : best_LGBM_model
         }

In [48]:
# создадим пустой список для последующего добавления в обзорную таблицу лучших моделей
f1_tests = []
                
# через цикл, обращаясь к словарю моделей, определим качество моделей по тестовой выборке
for name, model in models.items():
    test_predictions = model.predict(tf_idf_test)
    accuracy_test = round(f1_score(target_test, test_predictions), 3)
    f1_tests.append(accuracy_test)
    
    print(f'F1-мера модели {name} на тестовой выборке:', accuracy_test)
    print()
    
# создадим новый столбец в выводной таблице по лучшим моделям
models_summary['Качество при тесте'] = f1_tests

F1-мера модели Случайный лес на тестовой выборке: 0.508

F1-мера модели Логистическая регрессия с CV на тестовой выборке: 0.789

F1-мера модели Логистическая регрессия без CV на тестовой выборке: 0.788

F1-мера модели LightGBM на тестовой выборке: 0.741



Выведем еще раз итоговую таблицу

In [49]:
models_summary.sort_values(by='Качество при тесте', ascending=False)

,Модель,Лучшее качество F1-меры при обучении,"Время выполнения оптимального параметра, сек.",Лучшие гиперпараметры,Качество при тесте
1,Логистическая регрессия с CV,0.787425,385.77596,"['C' : 20, 'class_weight' : 'balanced']",0.789
2,Логистическая регрессия без CV,0.778610,60.79804,"['C' : 20, 'class_weight' : 'balanced']",0.788
3,LightGBM,0.736000,161.03665,"{'max_depth': 80, 'n_estimators': 86}",0.741
0,Случайный лес,0.495000,27.43883,"{'max_depth': 99, 'min_samples_split': 0.00329...",0.508


### Выводы

По итогу проверки на тестовых данных:
* модели Логистическая регрессия и LightGBM показали лучшее качество. К сожалению, XGBoost не удалось посмотреть из-за длительного перебора параметров ~7 часов в онлайн юпитере; потенциально там могли быть хорошие показатели, тк на момент остановки уже было 0.76, т.е. на тесте мы бы получили +/- .77-.78

## **IV. Проверка моделей на вменяемость**

Также проверим наши модели на адекватность сравнив их со случайными результатами, которые игнорируют правило входных данных. И если модель ведет себя лучше, то значит она может извлекать какой-то сигнал из данных и имеет предсказательную силу.
Для этого воспользуемся `sklearn.dummy.DummyClassifier`

In [50]:
X = tf_idf_test
y = target_test
dummy_reg = DummyClassifier(strategy="uniform")
dummy_reg.fit(X, y)
dummy_predictions = dummy_reg.predict(X)

print(f'F1-мера: {round(f1_score(y, dummy_predictions), 3)}')

F1-мера: 0.171


Наши лучшие модели показали качество лучше, чем случаная, следовательно имеет смысл использовать модели машинного обучения.

## **V. Итоги исследования**

### **Выводы**

В ходе подготовки данных текстовая часть была почищена и лемматизирована для анализа и дальнейшего обучения обучения. Также с целью выравнивания баланса классов были испробованы методы:
Также с целью выравнивания баланса классов были испробованы методы:
- **взвешивания классов `class_weight = 'balanced'`- вариант, на котором остановился**
- увеличения выборки редких значений путем их дублирования `upsample`
- уменьшения выборки путем удаления частых значений `downsample` 

На основании обработанных данных мы создали несколько моделей, выбрали лучшие, подобрали оптимальные гиперпараметры и проверили их на адекватность. И вот что у нас получилось:

In [51]:
models_summary.sort_values(by='Качество при тесте', ascending=False)

,Модель,Лучшее качество F1-меры при обучении,"Время выполнения оптимального параметра, сек.",Лучшие гиперпараметры,Качество при тесте
1,Логистическая регрессия с CV,0.787425,385.77596,"['C' : 20, 'class_weight' : 'balanced']",0.789
2,Логистическая регрессия без CV,0.778610,60.79804,"['C' : 20, 'class_weight' : 'balanced']",0.788
3,LightGBM,0.736000,161.03665,"{'max_depth': 80, 'n_estimators': 86}",0.741
0,Случайный лес,0.495000,27.43883,"{'max_depth': 99, 'min_samples_split': 0.00329...",0.508


Лучшей моделью для решения поставленной цели будет являться модель Логистическая регрессия. Кросс-валидация занимает слишком много времени, хоть и незначительно улучшает качество.